In [1]:
import customtkinter as ctk
import tkinter as tk
from tkinter.messagebox import showinfo, showerror
import pygame 
import re
import pygame_widgets
from pygame_widgets.button import Button

''' Класс отвечающий за окно ввода K, L, M. '''
class Menu_for_NLK(ctk.CTk):
    def __init__(self):
        super().__init__()
        self.resizable(False, False)
        self.title("Начальное меню")
        self.geometry(f"+{(self.winfo_screenwidth()//2 )-80}+{(self.winfo_screenheight()//2)-80}")

        self.all_value_true = [1, 1, 1]

        ctk.CTkLabel(self, text='Введите N размер поля').grid(row=0, column=0)
        self.entry_N = ctk.CTkEntry(self, validate='key', validatecommand=(self.register(self.validate_N), '%P'))
        self.entry_N.grid(row=0, column=1)

        ctk.CTkLabel(self, text='Введите L кол-во расставляемых фигур').grid(row=1, column=0)
        self.entry_L = ctk.CTkEntry(self, validate='key', validatecommand=(self.register(self.validate_L), '%P'))
        self.entry_L.grid(row=1, column=1)

        ctk.CTkLabel(self, text='Введите K кол-во размещённых фигур ').grid(row=2, column=0)
        self.entry_K = ctk.CTkEntry(self, validate='key', validatecommand=(self.register(self.validate_K), '%P'))
        self.entry_K.grid(row=2, column=1)

        ctk.CTkButton(self, text='Старт',width=60, command=self.button).grid(row=3,columnspan=2)

    ''' проверяет заполнены ли все значения, закрывает текущее окно и инициализирует окно для ввода координат'''
    def button(self):
        if sum(self.all_value_true) == 3:
            n = int(self.entry_N.get())
            k = int(self.entry_K.get())
            l = int(self.entry_L.get())
            self.destroy()
            Menu_for_all_K(n, l, k)

    ''' Проверяет правильность ввода N'''
    def validate_N(self, newval):
        if newval.isnumeric() and int(newval) <= 20 and len(newval) <= 2 and newval[0] != 0:
            self.all_value_true[0] = 1
            return True
        elif  newval == '':
            self.all_value_true[0] = 0
            return True
        else:
            showerror(title='Ошибка ввода', message='Значение N должно быть ЧИСЛОМ <=20')
            self.entry_N.delete(0, tk.END)
            return False

    ''' Проверяет правильность ввода L'''
    def validate_L(self, newval):
        if newval.isnumeric() and int(newval) <=80 and len(newval) <= 2:
            self.all_value_true[0] = 1
            return True
        elif newval == '':
            self.all_value_true[1] = 0
            return True
        else: 
            showerror(title='Ошибка ввода', message='Значение L должно быть ЧИСЛОМ <=80')
            self.entry_L.delete(0, tk.END)            
            return False

    ''' Проверяет правильность ввода K'''
    def validate_K(self, newval):
        if newval.isnumeric() and int(newval) <=80 and len(newval) <= 2:
            self.all_value_true[0] = 1
            return True
        elif newval == '':
            self.all_value_true[2] = 0
            return True
        else: 
            showerror(title='Ошибка ввода', message='Значение K должно быть ЧИСЛОМ <=80')
            self.entry_K.delete(0, tk.END)        
            return False

''' Класс отвечающий за окно ввода координат '''
class Menu_for_all_K(ctk.CTk):
    def __init__(self, n, l, k) -> None:
        super().__init__()
        self.n = n - 1
        self.l = l
        self.k = k

        self.resizable(False, False)
        self.title("Меню ввода координат")
        self.geometry(f"+{(self.winfo_screenwidth()//2 )-100}+{(self.winfo_screenheight()//2)-100}")

        self.scrollable_frame = ctk.CTkScrollableFrame(self, width=285, height=200)

        self.flag = False
        self.entry_figure = [] # список с ссылками на все k поля ввода координат
        self.data_figure = [] # список с координатами введенных фигур

        for i in range(self.k): # создаём фреймы Label, Entry для каждой из К фигур
            ctk.CTkLabel(self.scrollable_frame, text=f"Координаты {i+1} фигуры").grid(row=i, column=0)
            check = (self.register(self.validate), '%P')
            entry = ctk.CTkEntry(self.scrollable_frame, validate='key', validatecommand=check) 
            self.entry_figure.append(entry)
            entry.grid(row=i,column=1, padx=5)

        self.scrollable_frame.grid(row=0,column=0)

        ctk.CTkButton(self, text='Проверить ввод данных', width=60, command=self.button).grid(row=3,columnspan=2)
        ctk.CTkButton(self, text='Создать игровое поле', width=60, command=self.to_pygame).grid(row=4,columnspan=2)
        self.mainloop()

    ''' проверяет правильность введенных координат'''
    def button(self):
        flag = True
        self.occupied_kletki = []
        for i in self.entry_figure:
            if len(i.get()) != 3:
                flag = False
                showerror(title='Ошибка ввода', message='Были указаны координаты не всех фигур или указаны не полностью')
                break
            else:
                coords = [int(x) for x in i.get().split(' ')]
                curent_ocupied_kenki = self.F_occupied_kletki(self.n, coords)
                if coords in self.occupied_kletki:
                    flag = False
                    showerror(title='Ошибка ввода', message='Неверные координаты, фигуры попадают под бой друг друга')
                    break
                else:
                    self.occupied_kletki.extend(curent_ocupied_kenki)
                    self.data_figure.append(coords)

        if flag != False:  
            self.flag = True
            showinfo(title='Уведомление', message='Введенные данные коректны')

    ''' передаёт полученные координаты и инициализирует класс Actions, закрывает текущее окно'''
    def to_pygame(self):
        if self.flag == True:
            n,l,k = self.n, self.l, self.k
            ocup = self.occupied_kletki
            data_figure = self.data_figure
            self.destroy()
            Actions(n, l, k, ocup, data_figure)
        else:
            showerror(title='Ошибка ввода', message='Введенные координаты необходимо проверить')

    ''' проверяет введенные координаты'''
    def validate(self, newval):
        if newval == '' or re.match("\d{,2}\ {,1}\d{,2}$", newval)is not None:
            return True
        else: 
            showerror(title='Ошибка ввода', message='Неправильный формат ввода. Пример правильного: "1 1"')
            return False
        
    ''' находит все окупированные клетки для последней фигуры в списке
        получает на вход:
        n - размер поля
        data_fig - список координат фигур'''   
    @staticmethod
    def F_occupied_kletki(n: int, data_fig: list) -> list:
        occupied_kletki = []
        i=data_fig[0]
        j=data_fig[1]
        occupied_kletki.extend([data_fig])
        occupied_kletki.extend(list(filter(lambda x: 0<=x[0]<=n and 0<=x[1]<=n,[[i-1,j-1],[i-2,j-2],[i-3,j-3],[i-4,j-4]])))
        occupied_kletki.extend(list(filter(lambda x: 0<=x[0]<=n and 0<=x[1]<=n,[[i+1,j+1],[i+2,j+2],[i+3,j+3],[i+4,j+4]])))
        occupied_kletki.extend(list(filter(lambda x: 0<=x[0]<=n and 0<=x[1]<=n,[[i-1,j+1],[i-2,j+2],[i-3,j+3],[i-4,j+4]])))
        occupied_kletki.extend(list(filter(lambda x: 0<=x[0]<=n and 0<=x[1]<=n,[[i+1,j-1],[i+2,j-2],[i+3,j-3],[i+4,j-4]])))
        return occupied_kletki
    
''' класс содержащий в себе основные методы для работы с координатами фигур и игровой доской'''   
class Computing:
    def __init__(self) -> None:
        pass

    ''' находит все возможные варианты расстановки фигур
        n - размер поля
        l - сколько фигур необходимо расставить
        data_fig - координаты уже расставленных фигур
        occupied_kletki - координаты окупированные уже расставленными фигурами
        flag - позволяет получить или только первое возможное решение, если = [1,0] или все возможные решения, если = [0,0] (для оптимизации)
        x,y - текущие координаты (для оптимизации)'''
    def check(self, n: int, l: int, data_fig: list, occupied_kletki: list, flag, x, y) -> None:
        if l==0:
            dell = ''
            for i in range (len(data_fig)):
                dell = dell + f"{tuple(data_fig[i])}"
            self.all_rez.append(dell)
            if flag[0] == 1 and flag[1] == 0: # это условие говорит что нужно только первое решение, мы передаём его и завершаем поиск
                flag[1] = 1
                return [data_fig, occupied_kletki]
            return False # чтобы можно было вернуть предыдущие значения переменным во избежании ошибочных результатов (закрытие рекурсии)
        else:
            # проходимся по воображаемому полю, учитывая предыдущие координаты, дабы избежать повторений
            for i in range (x,n): 
                for j in range (n): 
                    if x==i and j<y: # пропускаем так как уже проверили
                        continue
                    else:
                        if ([i,j] in occupied_kletki): # пропускаем так как есть в окупированных клетках
                            pass
                        else:
                            data_fig = data_fig + [[i,j]] # добавляем новую фигуру
                            occupied_kletki_current = [] # для созранения предыдушего значения переменной occupied_kletki_dop, для правильной работы рекурсии
                            occupied_kletki_current.extend(occupied_kletki)
                            occupied_kletki_dop = self.F_occupied_kletki(n, data_fig)
                            occupied_kletki.extend(occupied_kletki_dop) # обновляем, добавляя окупированные координаты новой фигуры
                            if flag[0] == 0: # нужно найти все решения, поэтому возвращаем рекурсию на шаг назад
                                if not (self.check(n,l-1,data_fig,occupied_kletki,flag, i,j)):
                                    data_fig = data_fig[:-1]
                                    occupied_kletki = occupied_kletki_current
                            else:  # необходимо только одно решение
                                if l == 1: # если все фигуры расставлены, вызываем вновь для созранения решения
                                   self.check(n,0,data_fig,occupied_kletki,flag, i,j) 
                                   return [data_fig, occupied_kletki] # подстраховка чтобы рекурссия раотала правильно
                                l-=1 # уменьшаем значение l, т.к. уже жобавили новую фигуру

    ''' находит все окупированные клетки для последней фигуры в списке
        получает на вход:
        n - размер поля
        data_fig - список координат фигур'''   
    @staticmethod
    def F_occupied_kletki(n: int, data_fig: list) -> list:
        occupied_kletki = []
        i=data_fig[-1][0]
        j=data_fig[-1][1]
        occupied_kletki.extend([data_fig[-1]])
        occupied_kletki.extend(list(filter(lambda x: 0<=x[0]<=n and 0<=x[1]<=n,[[i-1,j-1],[i-2,j-2],[i-3,j-3],[i-4,j-4]])))
        occupied_kletki.extend(list(filter(lambda x: 0<=x[0]<=n and 0<=x[1]<=n,[[i+1,j+1],[i+2,j+2],[i+3,j+3],[i+4,j+4]])))
        occupied_kletki.extend(list(filter(lambda x: 0<=x[0]<=n and 0<=x[1]<=n,[[i-1,j+1],[i-2,j+2],[i-3,j+3],[i-4,j+4]])))
        occupied_kletki.extend(list(filter(lambda x: 0<=x[0]<=n and 0<=x[1]<=n,[[i+1,j-1],[i+2,j-2],[i+3,j-3],[i+4,j-4]])))
        return occupied_kletki

''' класс вызывает метод check из материнского класса и находит первое решение, которое передает классу Pygame_window для инициализации окна с игровым полем
        n - размер поля
        l - сколько фигур необходимо расставить
        k - количество уже расставленных фигур
        K_data_figure - координаты уже расставленных фигур
        k_occupied_kletki - координаты окупированные уже расставленными фигурами'''
class Actions(Computing):
    def __init__(self, n: int, l: int, k: int, k_occupied_kletki, K_data_figure) -> None:
        super().__init__()
        self.n = n 
        self.l = l
        self.k = k
        self.k_occupied_kletki = k_occupied_kletki
        self.K_data_figure = K_data_figure

        self.all_rez = []
        self.rez_occupied_kletki = []
        self.rez_data_figure = []
        self.flag = [0,0]

        self.flag = [1,0] # если первый элемент равен 1, значит находим первое решение и останавливаемся, иначе ищем все решения
        a = self.check(self.n+1, self.l, self.K_data_figure, self.k_occupied_kletki,self.flag, x=0, y=0 )
        if len(a) == 2: # проверка во избежании ошибок
            self.rez_output_data_figure, self.rez_output_occupied_kletki = a[0],a[1] # созраняем координаты фигур первого решения и их окупированные клетки
        self.flag = [0,0]
        if self.all_rez == []: # выводим ошибку, если не нейдено ни одно решение и закрываем окно
            showerror(title='Ошибка ввода', message='Не было найдено ни одного решения')
            self.destroy()
        self.all_rez = []

        # для создания игрового поля
        Pygame_window(self.n, self.l, self.k, self.K_data_figure, self.rez_output_data_figure, self.rez_output_occupied_kletki, self.all_rez)

''' отвечает за создание игрового поля и содержит кнопку для передачи всез решений в файл
        n - размер поля
        l - сколько фигур необходимо расставить
        k - количество уже расставленных фигур
        K_data_figure - координаты уже расставленных фигур
        k_occupied_kletki - координаты окупированные уже расставленными фигурами
        data_figure - координаты фигур, расставленных программой
        all_rez - все решения для вывода в файл'''
class Pygame_window(Computing):
    def __init__(self, n, l, k, K_data_figure, data_figure, occupied_kletki, all_rez) -> None:
        super().__init__()
        self.n = n+1
        self.l = l
        self.k = k
        self.K_data_figure = K_data_figure
        self.data_figure = data_figure
        self.occupied_kletki = occupied_kletki
        self.all_rez = all_rez

        black = (0, 0, 0)
        white = (255, 255, 255)
        mustard = (255,219,88) 
        melon_yellow = (244,169,0)
        yellow = (210,105,30)
        self.WIDTH = 30
        self.HEIGHT = 30
        self.MARGIN = 4

        pygame.init()
        self.side = self.n * (self.WIDTH + self.MARGIN + 0.6)
        self.scr = pygame.display.set_mode([self.side, self.side+int(self.side)//8])
        pygame.display.set_caption("Game")

        self.button = Button(self.scr,  int(self.side)//6, self.side+int(self.side)//50, self.side-2*int(self.side)//6, int(self.side)//13,
            text='Записать в файл',  # Текст кнопки
            fontSize=int(self.side)//12,  # Размер шрифта
            margin=2,  # Расстояние между текстом и кнопкой
            inactiveColour= white,  # Цвет неактивной кнопки
            hoverColour= yellow,  # Цвет кнопки после нажатия
            onClick=self.button)

        done = False
        while not done:
            events = pygame.event.get()
            for event in events: 
                if event.type == pygame.QUIT: 
                    done = True  
                    exit()

            self.scr.fill(black)
            for row in range(self.n): 
                for column in range(self.n):
                    color = white
                    if [row,column] in self.occupied_kletki :
                        color = mustard
                    if [row,column] in self.data_figure:
                        color = melon_yellow
                    if [row,column] in self.K_data_figure:
                        color = yellow

                    pygame.draw.rect(self.scr,
                                    color,
                                    [(self.MARGIN + self.WIDTH) * column + self.MARGIN,
                                    (self.MARGIN + self.HEIGHT) * row + self.MARGIN,
                                    self.WIDTH,
                                    self.HEIGHT])
                    
            pygame_widgets.update(events)
            pygame.display.update()
        pygame.quit()
    
    ''' отвечает за нахождение всех решений и передачу из в файл'''
    def button(self):
        occupied_kletki = []
        for i in self.K_data_figure:
            dell = self.F_occupied_kletki(self.n-1, [i])
            occupied_kletki.extend(dell)
        flag = [0,0]
        self.check(self.n, self.l, self.K_data_figure, occupied_kletki, flag, x=0,y=0 )
        for i in self.all_rez:
            f.write (f"{i}")
            f.write("\n")

if __name__ == '__main__':
    with open('output.txt', 'w') as f:
        app = Menu_for_NLK() 
        app.mainloop()


pygame 2.4.0 (SDL 2.26.4, Python 3.9.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


: 